# Library

In [75]:
import requests
from bs4 import BeautifulSoup as bs

import time, os, random, re
import pandas as pd
import numpy as np
from datetime import datetime

from selenium import webdriver
import chromedriver_autoinstaller as ca

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## chrome driver 설치

In [76]:
# USB error 메세지 발생 해결을 위한 코드
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])

In [77]:
# 현재 크롬 버전 확인
chrome_ver = ca.get_chrome_version().split('.')[0]
chrome_ver

'131'

In [4]:
# # 크롬 드라이버 확인 및 설치(처음 한번만 실행)
# ca.install(True)

# 페이지 접속

In [ ]:
# url = 'https://section.blog.naver.com/BlogHome.naver?directoryNo=0&currentPage=1&groupId=0' # 네이버 블로그 홈

## requests 테스트
- 페이지 접속 가능 여부확인
    - 가능할 경우 출력 : <Response [200]>

In [7]:
# req = requests.get(url)
# print(req)

# 한글 깨짐 해결 코드
# # html = req.content.decode('utf-8') # 한글 깨짐 해결
# # soup = bs(html, 'html.parser')

# soup = bs(req.text, 'html.parser')
# soup.title.text

<Response [200]>


'네이버 블로그'

In [78]:
# request로 수집 에러
keyword = '아이폰 16' # 검색어
page_num = 1 # 페이지 번호
rangetype = 'ALL' # 검색 범위
orderby = 'sim' # 정렬 순서

keyword_url = f'https://section.blog.naver.com/Search/Post.naver?pageNo={page_num}&rangeType={rangetype}&orderBy={orderby}&keyword={keyword}'
req = requests.get(keyword_url, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36'})
print(req)
soup = bs(req.text, 'html.parser')
soup.title.text

<Response [200]>


'네이버 블로그'

In [56]:
# area_list_search = soup.select_one('div.area_list_search')
# list_search_post = area_list_search.select('div.list_search_post')
# list_search_post[0]

## keyword 입력

In [79]:
keyword = '아이폰 16' # 검색어

## 탭 선택(글, 블로그)

In [80]:
tab_option = '글'
rangetype = 'PERIOD' # 검색 범위(기간 전체 : ALL, 최근 1주 : WEEK, 최근 1개월 : MONTH, 기간 입력 : PERIOD)
orderby = 'sim' # 정렬 순서(관련도순 : sim, 최신순 : recentdate)

# rangetype이 PERIOD인 경우 시작일과 종료일 설정 
startdate = '2025-01-01' # 형식: YYYY-mm-dd(예. 2025-01-01)
enddate = '2025-01-02' # 형식: YYYY-mm-dd(예. 2025-01-01)
current_date = datetime.today().strftime('%Y-%m-%d')
if startdate > enddate:
    startdate, enddate = enddate, startdate # 시작일이 종료일보다 크면 종료일로 변경
if startdate > current_date:
    startdate, enddate = current_date, current_date # 시작일이 현재 날짜보다 크면 현재 날짜로 변경
if enddate > current_date:
    enddate = current_date # 종료일이 현재 날짜보다 크면 현재 날짜로 변경

In [81]:
# 웹드라이버 실행
driver = webdriver.Chrome(options=options)

In [82]:
# 검색 결과 개수 확인
page_num = 1
if rangetype == 'PERIOD':
    keyword_url = f'https://section.blog.naver.com/Search/Post.naver?pageNo={page_num}&rangeType={rangetype}&orderBy={orderby}&&startDate={startdate}&endDate={enddate}&keyword={keyword}'
else:
    keyword_url = f'https://section.blog.naver.com/Search/Post.naver?pageNo={page_num}&rangeType={rangetype}&orderBy={orderby}&keyword={keyword}'

driver.get(keyword_url)
driver.implicitly_wait(10) # 페이지 로드 될 때까지 기다리지만 로드 되는 순간 종료
time.sleep(random.uniform(1, 3))

page = driver.page_source
soup = bs(page, 'html.parser')

area_list_search = soup.select_one('div.area_list_search')
list_search_post = area_list_search.select('div.list_search_post')

# 검색 결과 개수
raw_post_search_number = soup.select_one('div.search_information em.search_number').text
post_search_number = int(re.sub('[^0-9]', '', raw_post_search_number))

# 검색 결과 페이지 수
max_post_search_page_num = int(np.ceil(post_search_number / len(list_search_post)))

if rangetype == 'PERIOD':
    print('='*50)
    print('검색어 :', keyword)
    print('글/블로그 선택 :', tab_option)
    print('검색 범위 :', rangetype.replace('PERIOD', '기간 입력'))
    print('검색 시작일 :', startdate)
    print('검색 종료일 :', enddate)
    print('정렬 순서 :', orderby.replace('sim', '관련도순').replace('recentdate', '최신순'))
    print('검색 결과 개수 :', post_search_number)
    print('검색 결과 페이지 수 :', max_post_search_page_num)
else:
    print('검색어 :', keyword)
    print('글/블로그 선택 :', tab_option)
    print('검색 범위 :', rangetype.replace('ALL', '기간 전체').replace('WEEK', '최근 1주').replace('MONTH', '최근 1개월'))
    print('정렬 순서 :', orderby.replace('sim', '관련도순').replace('recentdate', '최신순'))
    print('검색 결과 개수 :', post_search_number)
    print('검색 결과 페이지 수 :', max_post_search_page_num)

검색어 : 아이폰 16
글/블로그 선택 : 글
검색 범위 : 기간 입력
검색 시작일 : 2025-01-01
검색 종료일 : 2025-01-02
정렬 순서 : 관련도순
검색 결과 개수 : 1065
검색 결과 페이지 수 : 153


# 정보 수집

### 글 선택
- 참고 링크 : https://section.blog.naver.com/Search/Post.naver?pageNo={page_num}&rangeType=ALL&orderBy=sim&keyword={keyword}
- 참고 링크 : https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=WEEK&orderBy=sim&startDate=2025-01-03&endDate=2025-01-10&keyword=아이폰 16
- 참고 링크 : https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=PERIOD&orderBy=sim&startDate=2025-01-01&endDate=2025-01-13&keyword=아이폰 16
- [⭕] rangeType 옵션(기간)
    - 기간전체 : &rangeType=ALL
    - 최근 1주 : &rangeType=WEEK
    - 최근 1개월 : &rangeType=MONTH
    - 기간 입력 : &rangeType=PERIOD&startDate=YYYY-mm-dd&endDate=YYYY-mm-dd

- [⭕] orderBy 옵션(관련도순, 최신순)
    - 관련도순 : &orderBy=sim
    - 최신순 : &orderBy=recentdate

- [⭕] 수집할 페이지 옵션 추가
- [⭕] 수집할 페이지가 실제 페이지 수보다 많을 경우 예외 처리

In [83]:
page = driver.page_source
soup = bs(page, 'html.parser')
print(soup.title.text)

네이버 블로그


In [84]:
area_list_search = soup.select_one('div.area_list_search')
list_search_post = area_list_search.select('div.list_search_post')
list_search_post[0]

<div anchor-text="content" bg-move-scroll-on-pop-state="" class="list_search_post" ng-repeat="post in postSearchCtrl.resultList">
<div bg-article-multi-img-style="" class="item multi_pic">
<div class="info_post" ng-class="{only_post: post.thumbnails.length == 0}">
<div class="desc">
<a bg-nclick="srs*l.text" class="desc_inner" href="https://blog.naver.com/sexyleep/223712553697" ng-href="https://blog.naver.com/sexyleep/223712553697" target="_blank">
<strong class="title_post">
<!-- ngIf: post.titleHeadColor == '#27C400' -->
<!-- ngIf: post.titleHeadColor == '#FF6565' -->
<span class="title" ng-bind-html="post.title"><strong class="search_keyword">아이폰16</strong>프로 성지 가격 할인 팁</span>
</strong>
</a><!-- ngIf: post.contents --><a bg-nclick="srs*l.text" class="text" href="https://blog.naver.com/sexyleep/223712553697" ng-bind-html="post.contents" ng-href="https://blog.naver.com/sexyleep/223712553697" ng-if="post.contents" target="_blank">특히 최신작인 <strong class="search_keyword">아이폰16</strong>프로는

In [85]:
len(list_search_post)

7

In [86]:
# 검색 결과 개수
raw_search_number = soup.select_one('div.search_information em.search_number').text
search_number = int(re.sub('[^0-9]', '', raw_search_number))
search_number

1065

In [87]:
# 검색 결과 페이지 수
max_search_page_num = int(np.ceil(search_number / len(list_search_post)))
max_search_page_num

153

In [88]:
# 글 제목
post_num = 0
title = list_search_post[post_num].select_one('span.title').text
title

'아이폰16프로 성지 가격 할인 팁'

In [89]:
# 블로그 내용
text = list_search_post[post_num].select_one('a.text').text
text

'특히 최신작인 아이폰16프로는 강력한 성능과 혁신적인 디자인으로 더욱 화제를 모으고 있습니다. 하지만 높은 가격이 걸림돌이 될 수 있는데요. 이번 글에서는 아이폰16프로를 경제적으로 구매할 수 있는 방법과 할인 혜택을 친근하게 알려드리겠습니다. 아이폰16프로 가격 목차 아이폰16프로... '

In [90]:
# 글 작성자
name_author = list_search_post[post_num].select_one('em.name_author').text
name_author

'테크매니아'

In [91]:
# 블로그 이름
name_blog = list_search_post[post_num].select_one('span.name_blog').text
name_blog

'테크톡톡'

In [92]:
# 글 작성 날짜
date = list_search_post[post_num].select_one('span.date').text
date

'2025. 1. 2.'

In [93]:
# 글 링크
desc_inner = list_search_post[post_num].select_one('a.desc_inner')['href']
desc_inner

'https://blog.naver.com/sexyleep/223712553697'

In [94]:
# 블로그 링크
author_blog_link = list_search_post[post_num].select_one('a.author')['href']
author_blog_link

'https://blog.naver.com/sexyleep'

In [95]:
list_search_post[1]

<div anchor-text="content" bg-move-scroll-on-pop-state="" class="list_search_post" ng-repeat="post in postSearchCtrl.resultList">
<div bg-article-multi-img-style="" class="item multi_pic">
<div class="info_post" ng-class="{only_post: post.thumbnails.length == 0}">
<div class="desc">
<a bg-nclick="srs*l.text" class="desc_inner" href="https://blog.naver.com/pangyolee/223713001880" ng-href="https://blog.naver.com/pangyolee/223713001880" target="_blank">
<strong class="title_post">
<!-- ngIf: post.titleHeadColor == '#27C400' -->
<!-- ngIf: post.titleHeadColor == '#FF6565' -->
<span class="title" ng-bind-html="post.title"><strong class="search_keyword">아이폰16</strong> 프로 차이와 성지 활용법으로 최적의 혜택 받기</span>
</strong>
</a><!-- ngIf: post.contents --><a bg-nclick="srs*l.text" class="text" href="https://blog.naver.com/pangyolee/223713001880" ng-bind-html="post.contents" ng-href="https://blog.naver.com/pangyolee/223713001880" ng-if="post.contents" target="_blank"><strong class="search_keyword">아이폰16</s

In [96]:
# 글 제목
post_num = 1
title = list_search_post[post_num].select_one('span.title').text
title

'아이폰16 프로 차이와 성지 활용법으로 최적의 혜택 받기'

In [97]:
# 블로그 내용
text = list_search_post[post_num].select_one('a.text').text
text

'아이폰16 시리즈는 강력한 성능과 고급스러운 디자인으로 많은 이들의 관심을 끌고 있습니다. 하지만 고가의 플래그십 모델인 만큼, 가격 부담이 크기 때문에 할인 혜택을 최대한 활용하는 것이 중요합니다. 이번 글에서는 아이폰16 프로 차이, 통신사 할인 비교, 성지 매장 활용법, 그리고... '

In [98]:
# 글 작성자
name_author = list_search_post[post_num].select_one('em.name_author').text
name_author

'DevelopPharmacy'

In [99]:
# 블로그 이름
name_blog = list_search_post[post_num].select_one('span.name_blog').text
name_blog

'DevelopPharmacy'

In [100]:
# 글 작성 날짜
date = list_search_post[post_num].select_one('span.date').text
date

'2025. 1. 2.'

In [101]:
# 글 링크
text_link = list_search_post[post_num].select_one('a.desc_inner')['href']
text_link

'https://blog.naver.com/pangyolee/223713001880'

In [102]:
start = 5
end = 3
start, end = end, start
print(start, end)

3 5


In [103]:
# 수집할 포스트 페이지 설정
while True:
    try:
        print(f'최대 포스트 페이지 수 : {max_post_search_page_num}')
        post_crawling_page_num = int(input(f"수집할 포스트 페이지 수를 입력하세요(숫자만 입력) : "))
        if post_crawling_page_num <= 0:
            print("0보다 큰 숫자를 입력하세요.")
            continue
        if post_crawling_page_num > max_post_search_page_num:
            print('입력한 숫자 :', post_crawling_page_num)
            print(f"입력한 숫자가 검색 결과 페이지 수보다 큽니다. {max_post_search_page_num}로 설정합니다.")
            post_crawling_page_num = max_post_search_page_num
        break
    except ValueError:
        print("유효한 숫자를 입력하세요.")

print(f"수집할 포스트 페이지 수: {post_crawling_page_num}")

최대 포스트 페이지 수 : 153
수집할 포스트 페이지 수: 5


In [104]:
keyword = '아이폰 16' # 검색어
tab_option = '글' # 포스트, 블로그 선택
rangetype = 'PERIOD' # 검색 범위(ALL, WEEK, MONTH, PERIOD)
orderby = 'sim' # 정렬 순서(sim, recentdate)

# rangetype이 PERIOD인 경우 시작일과 종료일 설정 
startdate = '2025-01-01' # 형식: YYYY-mm-dd(예. 2025-01-01)
enddate = '2025-01-10' # 형식: YYYY-mm-dd(예. 2025-01-01)
current_date = datetime.today().strftime('%Y-%m-%d')
if startdate > enddate:
    startdate, enddate = enddate, startdate # 시작일이 종료일보다 크면 종료일로 변경
if startdate > current_date:
    startdate, enddate = current_date, current_date # 시작일이 현재 날짜보다 크면 현재 날짜로 변경
if enddate > current_date:
    enddate = current_date # 종료일이 현재 날짜보다 크면 현재 날짜로 변경

title_list = []
text_list = []
name_author_list = []
name_blog_list = []
date_list = []
post_link_list = []
author_blog_link_list = []

for page_num in range(1, post_crawling_page_num + 1):
    if tab_option == '글':
        if rangetype == 'PERIOD':
            keyword_url = f'https://section.blog.naver.com/Search/Post.naver?pageNo={page_num}&rangeType={rangetype}&orderBy={orderby}&&startDate={startdate}&endDate={enddate}&keyword={keyword}'
        else:
            keyword_url = f'https://section.blog.naver.com/Search/Post.naver?pageNo={page_num}&rangeType={rangetype}&orderBy={orderby}&keyword={keyword}'
    elif tab_option == '블로그':
        keyword_url = f'https://section.blog.naver.com/Search/Blog.naver?pageNo={page_num}&orderBy={orderby}&keyword={keyword}'


    driver.get(keyword_url)
    driver.implicitly_wait(10) # 페이지 로드 될 때까지 기다리지만 로드 되는 순간 종료
    time.sleep(random.uniform(1, 3))

    page = driver.page_source
    soup = bs(page, 'html.parser')
    # print(soup.title.text)
    
    area_list_search = soup.select_one('div.area_list_search')
    list_search_post = area_list_search.select('div.list_search_post')

    for post in list_search_post:
        title = post.select_one('span.title').text
        text = post.select_one('a.text').text
        name_author = post.select_one('em.name_author').text
        name_blog = post.select_one('span.name_blog').text
        date = post.select_one('span.date').text
        post_link = post.select_one('a.desc_inner')['href']
        author_blog_link = post.select_one('a.author')['href']

        title_list.append(title)
        text_list.append(text)
        name_author_list.append(name_author)
        name_blog_list.append(name_blog)
        date_list.append(date)
        post_link_list.append(post_link)
        author_blog_link_list.append(author_blog_link)

print(len(title_list), len(text_list), len(name_author_list), len(name_blog_list), len(date_list), len(post_link_list), len(author_blog_link_list))

35 35 35 35 35 35 35


In [105]:
post_dict = {
    'title': title_list,
    'text': text_list,
    'name_author': name_author_list,
    'name_blog': name_blog_list,
    'date': date_list,
    'post_link': post_link_list,
    'author_blog_link': author_blog_link_list
}
post_df = pd.DataFrame(post_dict)
post_df.head()

,title,text,name_author,name_blog,date,post_link,author_blog_link
0,아이폰16 성능 15와 비교 및 할인팁,아이폰16 성능이 궁금하다면 이번 포스팅을 주목해 주세요. 전작 15 시리즈와 비교...,곰순이,곰순이,2025. 1. 10.,https://blog.naver.com/mlifeso/223722026004,https://blog.naver.com/mlifeso
1,아이폰16 프로 디자인 보조금 꿀팁까지,"오늘은 아이폰16 프로 디자인과 달라진 주요 스펙을 정리해 보고, 가격 할인을 받을...",얌못,경제꿈나무,2025. 1. 9.,https://blog.naver.com/cjfgns2/223720560043,https://blog.naver.com/cjfgns2
2,아이폰 16프로 내추럴 티타늄 언박싱 구매후기,아이폰 16프로 내추럴 티타늄 언박싱 구매후기 남편돈 남편산 구매후기입니다 저는 아...,grace,GRACE's lov diary,2025. 1. 7.,https://blog.naver.com/dmsp3535/223717372581,https://blog.naver.com/dmsp3535
3,아이폰16프로 256 가격 성지 할인,최신 스마트폰 중에서도 아이폰16 프로는 강력한 성능과 세련된 디자인으로 많은 이들...,테크매니아,테크톡톡,2025. 1. 3.,https://blog.naver.com/sexyleep/223712531240,https://blog.naver.com/sexyleep
4,아이폰16 프로 통신사 자급제 차이 가격 비교 가이드,오늘은 아이폰16 프로 통신사 자급제 차이에 대해 알아보겠습니다. 이동통신과 자급제...,IT 대장님,IT 전문대대,2025. 1. 9.,https://blog.naver.com/ykimceo/223720115490,https://blog.naver.com/ykimceo


In [106]:
new = post_df.drop_duplicates()
new.shape

(35, 7)

In [107]:
# 현재 날짜
current_date = datetime.today().strftime('%Y%m%d')
current_date

'20250114'

In [108]:
# 현재 경로 확인
code_path = os.getcwd().replace('\\', '/')
code_path

'c:/Users/kbjoo/Documents/Google_Drive/GitHub/my_projects/crawling/naver'

In [109]:
# 수집한 파일 저장할 폴더 생성
crawled_folder_path = os.path.join(code_path, 'crawled_data', 'naver_blog', current_date)
os.makedirs(crawled_folder_path, exist_ok=True)

In [110]:
current_datetime = datetime.today().strftime('%Y%m%d_%p_%I%M%S')
current_datetime

'20250114_PM_042129'

## 블로그 선택
- 참고 링크 : https://section.blog.naver.com/Search/Blog.naver?pageNo=1&orderBy=sim&keyword=아이폰 16
- 참고 링크 : https://section.blog.naver.com/Search/Blog.naver?pageNo=1&orderBy=recentdate&keyword=아이폰 16
- [⭕] orderBy 옵션
    - 관련도순 : &orderBy=sim
    - 최신순 : &orderBy=recentdate

- [⭕] 수집할 페이지 옵션 추가
- [⭕] 수집할 페이지가 실제 페이지 수보다 많을 경우 예외 처리

In [111]:
keyword = '아이폰 16' # 검색어
tab_option = '블로그' # 포스트, 블로그 선택
orderby = 'sim' # 정렬 순서(sim, recentdate)

if tab_option == '글':
    if rangetype == 'PERIOD':
        keyword_url = f'https://section.blog.naver.com/Search/Post.naver?pageNo={page_num}&rangeType={rangetype}&orderBy={orderby}&&startDate={startdate}&endDate={enddate}&keyword={keyword}'
    else:
        keyword_url = f'https://section.blog.naver.com/Search/Post.naver?pageNo={page_num}&rangeType={rangetype}&orderBy={orderby}&keyword={keyword}'
elif tab_option == '블로그':
    keyword_url = f'https://section.blog.naver.com/Search/Blog.naver?pageNo={page_num}&orderBy={orderby}&keyword={keyword}'

In [112]:
keyword_url

'https://section.blog.naver.com/Search/Blog.naver?pageNo=5&orderBy=sim&keyword=아이폰 16'

In [113]:
# 검색 결과 개수 확인
page_num = 1
keyword_url = f'https://section.blog.naver.com/Search/Blog.naver?pageNo={page_num}&orderBy={orderby}&keyword={keyword}'

driver.get(keyword_url)
driver.implicitly_wait(10) # 페이지 로드 될 때까지 기다리지만 로드 되는 순간 종료
time.sleep(random.uniform(1, 3))

page = driver.page_source
soup = bs(page, 'html.parser')

area_list_search = soup.select_one('div.area_list_search')
list_search_blog = area_list_search.select('div.list_search_blog')

# 검색 결과 개수
raw_blog_search_number = soup.select_one('div.search_information em.search_number').text
blog_search_number = int(re.sub('[^0-9]', '', raw_blog_search_number))

# 검색 결과 페이지 수
max_blog_search_page_num = int(np.ceil(blog_search_number / len(list_search_blog)))

print('='*50)
print('검색어 :', keyword)
print('글/블로그 선택 :', tab_option)
print('정렬 순서 :', orderby.replace('sim', '관련도순').replace('recentdate', '최신순'))
print('검색 결과 개수 :', blog_search_number)
print('검색 결과 페이지 수 :', max_blog_search_page_num)

검색어 : 아이폰 16
글/블로그 선택 : 블로그
정렬 순서 : 관련도순
검색 결과 개수 : 196
검색 결과 페이지 수 : 20


In [114]:
page = driver.page_source
soup = bs(page, 'html.parser')
print(soup.title.text)

네이버 블로그


In [115]:
area_list_search = soup.select_one('div.area_list_search')
list_search_blog = area_list_search.select('div.list_search_blog')
list_search_blog[0]

<div anchor-text="content" bg-move-scroll-on-pop-state="" class="list_search_blog" ng-repeat="blog in blogSearchCtrl.resultList">
<div class="item">
<a bg-nclick="srb*l.blogger" class="name_blog" href="https://blog.naver.com/qhrejddl7373" ng-href="https://blog.naver.com/qhrejddl7373" target="_blank">
<em class="text_blog" ng-bind-html="blog.blogName"><strong class="search_keyword">아이폰16</strong>프로사전예약혜택성지모아카페</em>
</a>
<p class="blog_intro" ng-bind-html="blog.blogDesc"><strong class="search_keyword">아이폰16</strong>프로사전예약혜택성지모아카페 정보 공유 블로그 입니다~</p>
<div class="writer_info">
<a bg-nclick="srb*l.blogger" class="author" href="https://blog.naver.com/qhrejddl7373" ng-href="https://blog.naver.com/qhrejddl7373" target="_blank">
<div class="thumbnail_author">
<img alt="블로거 썸네일" bg-image="https://blogpfthumb-phinf.pstatic.net/MjAyNDA5MTNfMTc3/MDAxNzI2MTk2NTg4Njgx.uAZNONyJWghf7L_Anmni-CY7bw7n1TYuESshHirSrCcg.d0j2fA62wXmE-SqYrDRIv206ty97Ju_CFz2HLa6oafEg.PNG/m2.png/m2.png?type=s1" class="img_author"

In [116]:
len(list_search_blog)

10

In [117]:
# 검색 결과 개수
raw_search_number = soup.select_one('div.search_information em.search_number').text
search_number = int(re.sub('[^0-9]', '', raw_search_number))
print(search_number)

# 검색 결과 페이지 수
max_search_page_num = int(np.ceil(search_number / len(list_search_blog)))
print(max_search_page_num)

196
20


In [118]:
# 블로그 제목
post_num = 0
text_blog = list_search_blog[post_num].select_one('em.text_blog').text
text_blog

'아이폰16프로사전예약혜택성지모아카페'

In [119]:
# 블로그 내용
blog_intro = list_search_blog[post_num].select_one('p.blog_intro').text
blog_intro

'아이폰16프로사전예약혜택성지모아카페 정보 공유 블로그 입니다~'

In [120]:
# 글 작성자
name_author = list_search_blog[post_num].select_one('em.name_author').text
name_author

'성지모아카페'

In [121]:
# 블로그 링크
blog_link = list_search_blog[post_num].select_one('a.name_blog')['href']
blog_link

'https://blog.naver.com/qhrejddl7373'

In [122]:
# 블로그 제목
post_num = 1
text_blog = list_search_blog[post_num].select_one('em.text_blog').text
text_blog

'아이폰16'

In [123]:
# 블로그 내용
blog_intro = list_search_blog[post_num].select_one('p.blog_intro').text
blog_intro

''

In [124]:
list_search_blog[post_num].select_one('p.blog_intro').text == ''

True

In [125]:
# 글 작성자
name_author = list_search_blog[post_num].select_one('em.name_author').text
name_author

'아이폰16'

In [126]:
# 블로그 링크
blog_link = list_search_blog[post_num].select_one('a.name_blog')['href']
blog_link

'https://blog.naver.com/crnzlwbrc'

In [127]:
# 수집할 블로그 페이지 설정
while True:
    try:
        print(f'최대 블로그 페이지 수 : {max_blog_search_page_num}')
        blog_crawling_page_num = int(input(f"수집할 블로그 페이지 수를 입력하세요(숫자만 입력) : "))
        if blog_crawling_page_num <= 0:
            print("0보다 큰 숫자를 입력하세요.")
            continue
        if blog_crawling_page_num > max_blog_search_page_num:
            print('입력한 숫자 :', blog_crawling_page_num)
            print(f"입력한 숫자가 검색 결과 페이지 수보다 큽니다. {max_blog_search_page_num}로 설정합니다.")
            blog_crawling_page_num = max_blog_search_page_num
        break
    except ValueError:
        print("유효한 숫자를 입력하세요.")

print(f"수집할 블로그 페이지 수: {blog_crawling_page_num}")

최대 블로그 페이지 수 : 20
수집할 블로그 페이지 수: 5


In [129]:
# 정보 수집
text_blog_list = []
blog_intro_list = []
name_author_list = []
blog_link_list = []
for page_num in range(1, blog_crawling_page_num + 1):
    keyword_url = f'https://section.blog.naver.com/Search/Blog.naver?pageNo={page_num}&orderBy={orderby}&keyword={keyword}'

    driver.get(keyword_url)
    driver.implicitly_wait(10) # 페이지 로드 될 때까지 기다리지만 로드 되는 순간 종료
    time.sleep(random.uniform(1, 3))

    page = driver.page_source
    soup = bs(page, 'html.parser')
    # print(soup.title.text)

    area_list_search = soup.select_one('div.area_list_search')
    list_search_blog = area_list_search.select('div.list_search_blog')

    for blog in list_search_blog:
        text_blog = blog.select_one('em.text_blog').text
        if blog.select_one('p.blog_intro').text == '':
            blog_intro = np.nan
        else:
            blog_intro = blog.select_one('p.blog_intro').text
        name_author = blog.select_one('em.name_author').text
        blog_link = blog.select_one('a.name_blog')['href']
        # print(text_blog, blog_intro, name_author, blog_link)

        text_blog_list.append(text_blog)
        blog_intro_list.append(blog_intro)
        name_author_list.append(name_author)
        blog_link_list.append(blog_link)

print(len(text_blog_list), len(blog_intro_list), len(name_author_list), len(blog_link_list))

50 50 50 50


In [131]:
blog_dict = {
    'text_blog': text_blog_list,
    'blog_intro': blog_intro_list,
    'name_author': name_author_list,
    'blog_link': blog_link_list
}
blog_df = pd.DataFrame(blog_dict)
blog_df.head()

,text_blog,blog_intro,name_author,blog_link
0,아이폰16프로사전예약혜택성지모아카페,아이폰16프로사전예약혜택성지모아카페 정보 공유 블로그 입니다~,성지모아카페,https://blog.naver.com/qhrejddl7373
1,아이폰16,NaN,아이폰16,https://blog.naver.com/crnzlwbrc
2,탐방,아이폰 16 무보정 현실적인 탐방리뷰,탐방,https://blog.naver.com/hyeonmin1356
3,"먹고, 쉬고.","맛집, 카페, 여행 [촬영 : 아이폰 16프로]",잠만보,https://blog.naver.com/unicodin
4,먹짱일기장,아이폰 16 pro max오너의 맛도리 블로그,유댕,https://blog.naver.com/eliyoujin


## 종합

In [4]:
keyword = '아이폰 16' # 검색어

In [13]:
rangetype = 'PERIOD' # 검색 범위(기간 전체 : ALL, 최근 1주 : WEEK, 최근 1개월 : MONTH, 기간 입력 : PERIOD)
orderby = 'sim' # 정렬 순서(관련도순 : sim, 최신순 : recentdate)

# rangetype이 PERIOD인 경우 시작일과 종료일 설정 
startdate = '2025-01-01' # 형식: YYYY-mm-dd(예. 2025-01-01)
enddate = '2025-01-02' # 형식: YYYY-mm-dd(예. 2025-01-01)
current_date = datetime.today().strftime('%Y-%m-%d')
if startdate > enddate:
    startdate, enddate = enddate, startdate # 시작일이 종료일보다 크면 종료일로 변경
if startdate > current_date:
    startdate, enddate = current_date, current_date # 시작일이 현재 날짜보다 크면 현재 날짜로 변경
if enddate > current_date:
    enddate = current_date # 종료일이 현재 날짜보다 크면 현재 날짜로 변경

In [7]:
# 웹드라이버 실행
driver = webdriver.Chrome(options=options)

In [14]:
# 검색 결과 개수 확인
sheet_name_list = ['글', '블로그']
for tab_option in sheet_name_list:
    page_num = 1
    if tab_option == '글':
        if rangetype == 'PERIOD':
            keyword_url = f'https://section.blog.naver.com/Search/Post.naver?pageNo={page_num}&rangeType={rangetype}&orderBy={orderby}&&startDate={startdate}&endDate={enddate}&keyword={keyword}'
        else:
            keyword_url = f'https://section.blog.naver.com/Search/Post.naver?pageNo={page_num}&rangeType={rangetype}&orderBy={orderby}&keyword={keyword}'

        driver.get(keyword_url)
        driver.implicitly_wait(10) # 페이지 로드 될 때까지 기다리지만 로드 되는 순간 종료
        time.sleep(random.uniform(1, 3))

        page = driver.page_source
        soup = bs(page, 'html.parser')
        
        area_list_search = soup.select_one('div.area_list_search')
        list_search_post = area_list_search.select('div.list_search_post')

        # 검색 결과 개수
        raw_post_search_number = soup.select_one('div.search_information em.search_number').text
        post_search_number = int(re.sub('[^0-9]', '', raw_post_search_number))

        # 검색 결과 페이지 수
        max_post_search_page_num = int(np.ceil(post_search_number / len(list_search_post)))

        if rangetype == 'PERIOD':
            print('='*50)
            print('검색어 :', keyword)
            print('글/블로그 선택 :', tab_option)
            print('검색 범위 :', rangetype.replace('PERIOD', '기간 입력'))
            print('검색 시작일 :', startdate)
            print('검색 종료일 :', enddate)
            print('정렬 순서 :', orderby.replace('sim', '관련도순').replace('recentdate', '최신순'))
            print('검색 결과 개수 :', post_search_number)
            print('검색 결과 페이지 수 :', max_post_search_page_num)
        else:
            print('검색어 :', keyword)
            print('글/블로그 선택 :', tab_option)
            print('검색 범위 :', rangetype.replace('ALL', '기간 전체').replace('WEEK', '최근 1주').replace('MONTH', '최근 1개월'))
            print('정렬 순서 :', orderby.replace('sim', '관련도순').replace('recentdate', '최신순'))
            print('검색 결과 개수 :', post_search_number)
            print('검색 결과 페이지 수 :', max_post_search_page_num)
        print()

    elif tab_option == '블로그':
        keyword_url = f'https://section.blog.naver.com/Search/Blog.naver?pageNo={page_num}&orderBy={orderby}&keyword={keyword}'

        driver.get(keyword_url)
        driver.implicitly_wait(10) # 페이지 로드 될 때까지 기다리지만 로드 되는 순간 종료
        time.sleep(random.uniform(1, 3))

        page = driver.page_source
        soup = bs(page, 'html.parser')

        area_list_search = soup.select_one('div.area_list_search')
        list_search_blog = area_list_search.select('div.list_search_blog')

        # 검색 결과 개수
        raw_blog_search_number = soup.select_one('div.search_information em.search_number').text
        blog_search_number = int(re.sub('[^0-9]', '', raw_blog_search_number))

        # 검색 결과 페이지 수
        max_blog_search_page_num = int(np.ceil(blog_search_number / len(list_search_blog)))
        
        print('='*50)
        print('검색어 :', keyword)
        print('글/블로그 선택 :', tab_option)
        print('정렬 순서 :', orderby.replace('sim', '관련도순').replace('recentdate', '최신순'))
        print('검색 결과 개수 :', blog_search_number)
        print('검색 결과 페이지 수 :', max_blog_search_page_num)

검색어 : 아이폰 16
글/블로그 선택 : 글
검색 범위 : 기간 입력
검색 시작일 : 2025-01-01
검색 종료일 : 2025-01-02
정렬 순서 : 관련도순
검색 결과 개수 : 1067
검색 결과 페이지 수 : 153

검색어 : 아이폰 16
글/블로그 선택 : 블로그
정렬 순서 : 관련도순
검색 결과 개수 : 196
검색 결과 페이지 수 : 20


In [15]:
# 수집할 포스트 페이지 설정
while True:
    try:
        print(f'최대 포스트 페이지 수 : {max_post_search_page_num}')
        post_crawling_page_num = int(input(f"수집할 포스트 페이지 수를 입력하세요(숫자만 입력) : "))
        if post_crawling_page_num <= 0:
            print("0보다 큰 숫자를 입력하세요.")
            continue
        if post_crawling_page_num > max_post_search_page_num:
            print('입력한 숫자 :', post_crawling_page_num)
            print(f"입력한 숫자가 검색 결과 페이지 수보다 큽니다. {max_post_search_page_num}로 설정합니다.")
            post_crawling_page_num = max_post_search_page_num
        break
    except ValueError:
        print("유효한 숫자를 입력하세요.")

print(f"수집할 포스트 페이지 수: {post_crawling_page_num}")

최대 포스트 페이지 수 : 153
수집할 포스트 페이지 수: 5


In [16]:
# 수집할 블로그 페이지 설정
while True:
    try:
        print(f'최대 블로그 페이지 수 : {max_blog_search_page_num}')
        blog_crawling_page_num = int(input(f"수집할 블로그 페이지 수를 입력하세요(숫자만 입력) : "))
        if blog_crawling_page_num <= 0:
            print("0보다 큰 숫자를 입력하세요.")
            continue
        if blog_crawling_page_num > max_blog_search_page_num:
            print('입력한 숫자 :', blog_crawling_page_num)
            print(f"입력한 숫자가 검색 결과 페이지 수보다 큽니다. {max_blog_search_page_num}로 설정합니다.")
            blog_crawling_page_num = max_blog_search_page_num
        break
    except ValueError:
        print("유효한 숫자를 입력하세요.")

print(f"수집할 블로그 페이지 수: {blog_crawling_page_num}")

최대 블로그 페이지 수 : 20
수집할 블로그 페이지 수: 5


In [17]:
# 현재 날짜
current_date = datetime.today().strftime('%Y%m%d')
# 현재 경로 확인
code_path = os.getcwd().replace('\\', '/')
# 수집한 파일 저장할 폴더 생성
crawled_folder_path = os.path.join(code_path, 'crawled_data', 'naver_blog', current_date)
os.makedirs(crawled_folder_path, exist_ok=True)
# 저장할 파일 경로
current_datetime = datetime.today().strftime('%Y%m%d_%p_%I%M%S')
file_path = os.path.join(crawled_folder_path, f'naver_blog_{current_datetime}.xlsx')

In [ ]:
# 페이지 수집
sheet_name_list = ['글', '블로그']
with pd.ExcelWriter(file_path, engine='openpyxl') as writer:
    for tab_option in sheet_name_list:
        if tab_option == '글':
            title_list = []
            text_list = []
            name_author_list = []
            name_blog_list = []
            date_list = []
            post_link_list = []
            author_blog_link_list = []

            for page_num in range(1, post_crawling_page_num + 1):
                if rangetype == 'PERIOD':
                    keyword_url = f'https://section.blog.naver.com/Search/Post.naver?pageNo={page_num}&rangeType={rangetype}&orderBy={orderby}&&startDate={startdate}&endDate={enddate}&keyword={keyword}'
                else:
                    keyword_url = f'https://section.blog.naver.com/Search/Post.naver?pageNo={page_num}&rangeType={rangetype}&orderBy={orderby}&keyword={keyword}'

                driver.get(keyword_url)
                driver.implicitly_wait(10) # 페이지 로드 될 때까지 기다리지만 로드 되는 순간 종료
                time.sleep(random.uniform(1, 3))

                page = driver.page_source
                soup = bs(page, 'html.parser')
                # print(soup.title.text)
                
                area_list_search = soup.select_one('div.area_list_search')
                list_search_post = area_list_search.select('div.list_search_post')

                for post in list_search_post:
                    title = post.select_one('span.title').text
                    text = post.select_one('a.text').text
                    name_author = post.select_one('em.name_author').text
                    name_blog = post.select_one('span.name_blog').text
                    date = post.select_one('span.date').text
                    post_link = post.select_one('a.desc_inner')['href']
                    author_blog_link = post.select_one('a.author')['href']

                    title_list.append(title)
                    text_list.append(text)
                    name_author_list.append(name_author)
                    name_blog_list.append(name_blog)
                    date_list.append(date)
                    post_link_list.append(post_link)
                    author_blog_link_list.append(author_blog_link)

            # print(len(title_list), len(text_list), len(name_author_list), len(name_blog_list), len(date_list), len(post_link_list), len(author_blog_link_list))

            # 데이터 프레임 생성
            post_dict = {
                'title': title_list,
                'text': text_list,
                'name_author': name_author_list,
                'name_blog': name_blog_list,
                'date': date_list,
                'post_link': post_link_list,
                'author_blog_link': author_blog_link_list
            }
            post_df = pd.DataFrame(post_dict)
            post_df.to_excel(writer, sheet_name=tab_option, index=False)
            print(post_df.shape)

        elif tab_option == '블로그':
            text_blog_list = []
            blog_intro_list = []
            name_author_list = []
            blog_link_list = []
            for page_num in range(1, blog_crawling_page_num + 1):
                keyword_url = f'https://section.blog.naver.com/Search/Blog.naver?pageNo={page_num}&orderBy={orderby}&keyword={keyword}'

                driver.get(keyword_url)
                driver.implicitly_wait(10) # 페이지 로드 될 때까지 기다리지만 로드 되는 순간 종료
                time.sleep(random.uniform(1, 3))

                page = driver.page_source
                soup = bs(page, 'html.parser')
                # print(soup.title.text)

                area_list_search = soup.select_one('div.area_list_search')
                list_search_blog = area_list_search.select('div.list_search_blog')

                for blog in list_search_blog:
                    text_blog = blog.select_one('em.text_blog').text
                    if blog.select_one('p.blog_intro').text == '':
                        blog_intro = np.nan
                    else:
                        blog_intro = blog.select_one('p.blog_intro').text
                    name_author = blog.select_one('em.name_author').text
                    blog_link = blog.select_one('a.name_blog')['href']
                    # print(text_blog, blog_intro, name_author, blog_link)

                    text_blog_list.append(text_blog)
                    blog_intro_list.append(blog_intro)
                    name_author_list.append(name_author)
                    blog_link_list.append(blog_link)

                # print(len(text_blog_list), len(blog_intro_list), len(name_author_list), len(blog_link_list))
            blog_dict = {
                'text_blog': text_blog_list,
                'blog_intro': blog_intro_list,
                'name_author': name_author_list,
                'blog_link': blog_link_list
            }
            blog_df = pd.DataFrame(blog_dict)
            blog_df.to_excel(writer, sheet_name=tab_option, index=False)
            print(blog_df.shape)

    print('저장 파일 경로 :', file_path)
    print('저장완료')

(35, 6)
(50, 4)
저장 파일 경로 : c:/Users/kbjoo/Documents/Google_Drive/GitHub/my_projects/crawling/naver\crawled_data\naver_blog\20250114\naver_blog_20250114_PM_035009.xlsx
저장완료


In [132]:
driver.quit()

# END